In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sopt
from pysimu import ode2numba, ssa

from ipywidgets import *


In [3]:
%matplotlib notebook

## System definition 

In [17]:
S_base = 100.0e6
U_base = 20e3
Z_base = U_base**2/S_base
r_km = 0.127 # ohm/km
x_km = 0.113 # ohm/km
lenght = 1.0
R = r_km*lenght/Z_base
X = x_km*lenght/Z_base
Z = R +1j*X
Y = 1.0/Z
G_s_inf, B_s_inf = Y.real, Y.imag

sys = { 't_end':20.0,'Dt':0.01,'solver':'forward-euler', 'decimation':10, 'name':'vsg_monk',
   'models':[{'params':
                   {'H':6.0,
                    'D':20,
                    'X_s': 0.1,
                    'R_s': 0.0,
                    'K_q':1.0,
                    'T_q':1.0,
                    'Omega_b' : 2*np.pi*60,
                    'R_g':0.01,
                    'X_g':0.05,
                    #'phi_g': 0.0, 
                    'V_g': 1.0,
                    'K_pll':0.01592
                   },
              'f':[
                'ddelta = Omega_b*(omega_t-1)',
                'domega_t = 1/(2*H)*(p_m - p_e - (omega_t-(omega_pll+1))*D)',
                'dxi_q = epsilon_q',
                'dphi_pll = Omega_b*omega_pll',
                'dphi_g = Omega_b*(omega_g - 1) -1e-5*phi_g',
                'domega_g = RoCoFpu - 1e-5*(omega_g - 1)'
                  ],   
              'g':[
                   'p_h@ p_h + 2*H*RoCoFpu',
                   'p_m0 @ p_m0 - p_m',
                   'v_gr@-v_gr + V_g*cos(phi_g)',
                   'v_gi@-v_gi + V_g*sin(phi_g)',
                   'i_sr @ - i_sr -(R_g*(v_sr - v_gr) + X_g*(v_si -v_gi))/(X_g**2 + R_g**2)', # kron reference frame
                   'i_si @ - i_si -(R_g*(v_si - v_gi) - X_g*(v_sr -v_gr))/(X_g**2 + R_g**2)', # kron reference frame
                   'i_sd_s@-i_sd_s + i_si', # phasor to dq in POI reference frame (v_sd_s = 0)
                   'i_sq_s@-i_sq_s - i_sr', # phasor to dq                   
                   'i_sd @-i_sd + cos(delta)*i_sd_s + cos(delta-pi/2)*i_sq_s',# virtual rotor reference frame
                   'i_sq @-i_sq - sin(delta)*i_sd_s - sin(delta-pi/2)*i_sq_s',# virtual rotor reference frame
                   'v_sd @ v_sd - v_td - R_s*i_sd + X_s*i_sq',# virtual rotor reference frame
                   'v_sq @ v_sq - v_tq - R_s*i_sq - X_s*i_sd',# virtual rotor reference frame
                   'v_sd_s @ -v_sd_s + cos(-delta)*v_sd + cos(-delta-pi/2)*v_sq', # kron reference frame
                   'v_sq_s @ -v_sq_s - sin(-delta)*v_sd - sin(-delta-pi/2)*v_sq', # kron reference frame
                   'v_si@-v_sd_s + v_si', # dq to phasor  
                   'v_sr@-v_sq_s - v_sr', # dq to phasor     
                   'epsilon_q@-epsilon_q + q_s_ref - q_s',
                   'e    @ -e -1+ K_q*(epsilon_q + xi_q/T_q) ', # 
                   'v_td@ -v_td + 0.0', # virtual rotor reference frame
                   'v_tq@ -v_tq +e', # virtual rotor reference frame
                   'p_e@-p_e+ i_sd*v_td + i_sq*v_tq', # inetenal active power equation 
                   'p_s@-p_s+ i_sd*v_sd + i_sq*v_sq', # active power equation 
                   'q_s@-q_s+ i_sd*v_sq - i_sq*v_sd', # reactive power equation 
                   'v_sd_pll@ -v_sd_pll + cos(phi_pll)*v_sr + cos(phi_pll-pi/2)*v_si',
                   'v_sq_pll@ -v_sq_pll - sin(phi_pll)*v_sr - sin(phi_pll-pi/2)*v_si', 
                   'omega_pll@-omega_pll + K_pll*v_sq_pll'
                   ],
              'u':{'p_m':0.8,'q_s_ref':0.1, 'RoCoFpu':0.0},
              'y':['p_h','p_m0','v_gr','v_gi','i_sr', 'i_si', 'i_sd_s', 'i_sq_s', 'i_sd','i_sq','v_sd','v_sq','v_sd_s','v_sq_s',
                   'v_sr','v_si','epsilon_q','e','v_td','v_tq','p_e','p_s','q_s','v_sd_pll','v_sq_pll','omega_pll'],
          'y_ini':['p_h','p_m0','v_gr','v_gi','i_sr', 'i_si', 'i_sd_s', 'i_sq_s', 'i_sd','i_sq','v_sd','v_sq','v_sd_s','v_sq_s',
                   'v_sr','v_si','epsilon_q','e','v_td','v_tq','p_e','p_s','q_s','v_sd_pll','v_sq_pll','omega_pll'],
              'h':[
                   'omega_t'
                   ]}
              ],
    'perturbations':[{'type':'step','time':100.0,'var':'p_m','final':1.01} ]
    }

x,f = ode2numba.system(sys)  ;


p_h
p_m0 
v_gr
v_gi
i_sr 
i_si 
i_sd_s
i_sq_s
i_sd 
i_sq 
v_sd 
v_sq 
v_sd_s 
v_sq_s 
v_si
v_sr
epsilon_q
e    
v_td
v_tq
p_e
p_s
q_s
v_sd_pll
v_sq_pll
omega_pll
 p_h + 2*H*RoCoFpu,  p_m0 - p_m, -v_gr + V_g*cos(phi_g), -v_gi + V_g*sin(phi_g),  - i_sr -(R_g*(v_sr - v_gr) + X_g*(v_si -v_gi))/(X_g**2 + R_g**2),  - i_si -(R_g*(v_si - v_gi) - X_g*(v_sr -v_gr))/(X_g**2 + R_g**2), -i_sd_s + i_si, -i_sq_s - i_sr, -i_sd + cos(delta)*i_sd_s + cos(delta-pi/2)*i_sq_s, -i_sq - sin(delta)*i_sd_s - sin(delta-pi/2)*i_sq_s,  v_sd - v_td - R_s*i_sd + X_s*i_sq,  v_sq - v_tq - R_s*i_sq - X_s*i_sd,  -v_sd_s + cos(-delta)*v_sd + cos(-delta-pi/2)*v_sq,  -v_sq_s - sin(-delta)*v_sd - sin(-delta-pi/2)*v_sq, -v_sd_s + v_si, -v_sq_s - v_sr, -epsilon_q + q_s_ref - q_s,  -e -1+ K_q*(epsilon_q + xi_q/T_q) ,  -v_td + 0.0,  -v_tq +e, -p_e+ i_sd*v_td + i_sq*v_tq, -p_s+ i_sd*v_sd + i_sq*v_sq, -q_s+ i_sd*v_sq - i_sq*v_sd,  -v_sd_pll + cos(phi_pll)*v_sr + cos(phi_pll-pi/2)*v_si,  -v_sq_pll - sin(phi_pll)*v_sr - sin(phi_pl